This cell is for loading the data and installing relevant librarys

In [6]:
import pandas as pd
import numpy as np
file = ('/Users/olisa/Downloads/clean_df.csv') # convert to websit link later
data = pd.read_csv(file)
data.head()

,id,listing_url,name,description,neighborhood_overview,host_id,host_url,host_name,host_since,host_location,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,54730.0,https://www.airbnb.com/users/show/54730,Alina,2009-11-16,"London, United Kingdom",...,4.86,4.93,4.75,4.82,f,2.0,1.0,1.0,0.0,0.20
1,15400,https://www.airbnb.com/rooms/15400,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,60302.0,https://www.airbnb.com/users/show/60302,Philippa,2009-12-05,"Kensington, United Kingdom",...,4.87,4.82,4.93,4.73,f,1.0,1.0,0.0,0.0,0.57
2,284532,https://www.airbnb.com/rooms/284532,COSY STUDIO-FLAT WITH A GREAT VIEW,My very RECENTLY RENOVATED flat is located in ...,Shoreditch is simply the coolest area in London!,1481412.0,https://www.airbnb.com/users/show/1481412,Luca,2011-12-07,"London, United Kingdom",...,4.72,4.86,4.72,4.31,f,1.0,1.0,0.0,0.0,0.34
3,106332,https://www.airbnb.com/rooms/106332,"Lovely large room, Bethnal Green",Lovely characterful room in a house belonging ...,NaN,264345.0,https://www.airbnb.com/users/show/264345,Susie,2010-10-17,"London, United Kingdom",...,4.93,4.93,4.92,4.83,f,1.0,0.0,1.0,0.0,0.47
4,17402,https://www.airbnb.com/rooms/17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",67564.0,https://www.airbnb.com/users/show/67564,Liz,2010-01-04,"London, United Kingdom",...,4.69,4.69,4.88,4.59,f,5.0,5.0,0.0,0.0,0.36


Next step is cleaning the dataframe and removing the colums that are not neccsary for the analysis.

In [12]:
columns_to_drop = ['id', 'listing_url', 'description', 'neighborhood_overview', 'host_id', 'host_url', 'host_about', 'host_picture_url', 'host_verifications', 'host_identity_verified', 'neighbourhood_cleansed', 'bathrooms_text', 'amenities', 'first_review', 'last_review', 'instant_bookable']
data.drop(columns = columns_to_drop, inplace=True)
data.head()

,name,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,Holiday London DB Room Let-on going,Alina,2009-11-16,"London, United Kingdom",within a day,80%,70%,f,3.0,4.0,...,4.89,4.86,4.93,4.75,4.82,2.0,1.0,1.0,0.0,0.20
1,Bright Chelsea Apartment. Chelsea!,Philippa,2009-12-05,"Kensington, United Kingdom",within an hour,67%,29%,f,1.0,12.0,...,4.88,4.87,4.82,4.93,4.73,1.0,1.0,0.0,0.0,0.57
2,COSY STUDIO-FLAT WITH A GREAT VIEW,Luca,2011-12-07,"London, United Kingdom",within a day,90%,67%,f,1.0,3.0,...,4.03,4.72,4.86,4.72,4.31,1.0,1.0,0.0,0.0,0.34
3,"Lovely large room, Bethnal Green",Susie,2010-10-17,"London, United Kingdom",within a few hours,100%,73%,t,2.0,5.0,...,4.71,4.93,4.93,4.92,4.83,1.0,0.0,1.0,0.0,0.47
4,Superb 3-Bed/2 Bath & Wifi: Trendy W1,Liz,2010-01-04,"London, United Kingdom",within a few hours,90%,87%,f,6.0,32.0,...,4.69,4.69,4.69,4.88,4.59,5.0,5.0,0.0,0.0,0.36


In [13]:
#checking that the right amount of columns have been deleted 
num_columns = len(data.columns)
num_columns

36

In [14]:
data.columns

Index(['name', 'host_name', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')